# ML Models

## Import Libraries & Load Dataframe from AWS DB

In [59]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import preprocessing
from math import sin, cos, sqrt, atan2, radians


In [213]:
#start by making dict of grades/ratings
diff_grade = {'Rope': ['3rd',
  '4th',
  'Easy 5th',
  '5.0',
  '5.1',
  '5.2',
  '5.3',
  '5.4',
  '5.5',
  '5.6',
  '5.7',
  '5.7+',
  '5.8-',
  '5.8',
  '5.8+',
  '5.9-',
  '5.9',
  '5.9+',
  '5.10a',
  '5.10-',
  '5.10a/b',
  '5.10b',
  '5.10',
  '5.10b/c',
  '5.10c',
  '5.10+',
  '5.10c/d',
  '5.10d',
  '5.11a',
  '5.11-',
  '5.11a/b',
  '5.11b',
  '5.11',
  '5.11b/c',
  '5.11c',
  '5.11+',
  '5.11c/d',
  '5.11d',
  '5.12a',
  '5.12-',
  '5.12a/b',
  '5.12b',
  '5.12',
  '5.12b/c',
  '5.12c',
  '5.12+',
  '5.12c/d',
  '5.12d',
  '5.13a',
  '5.13-',
  '5.13a/b',
  '5.13b',
  '5.13',
  '5.13b/c',
  '5.13c',
  '5.13+',
  '5.13c/d',
  '5.13d',
  '5.14a',
  '5.14-',
  '5.14a/b',
  '5.14b',
  '5.14',
  '5.14b/c',
  '5.14c',
  '5.14+',
  '5.14c/d',
  '5.14d',
  '5.15a',
  '5.15-',
  '5.15a/b',
  '5.15b',
  '5.15',
  '5.15c',
  '5.15+',
  '5.15c/d',
  '5.15d'],
 'Boulder': ['V-easy',
  'V0-',
  'V0',
  'V0+',
  'V0-1',
  'V1-',
  'V1',
  'V1+',
  'V1-2',
  'V2-',
  'V2',
  'V2+',
  'V2-3',
  'V3-',
  'V3',
  'V3+',
  'V3-4',
  'V4-',
  'V4',
  'V4+',
  'V4-5',
  'V5-',
  'V5',
  'V5+',
  'V5-6',
  'V6-',
  'V6',
  'V6+',
  'V6-7',
  'V7-',
  'V7',
  'V7+',
  'V7-8',
  'V8-',
  'V8',
  'V8+',
  'V8-9',
  'V9-',
  'V9',
  'V9+',
  'V9-10',
  'V10-',
  'V10',
  'V10+',
  'V10-11',
  'V11-',
  'V11',
  'V11+',
  'V11-12',
  'V12-',
  'V12',
  'V12+',
  'V12-13',
  'V13-',
  'V13',
  'V13+',
  'V13-14',
  'V14-',
  'V14',
  'V14+',
  'V14-15',
  'V15-',
  'V15',
  'V15+',
  'V15-16',
  'V16-',
  'V16',
  'V16+',
  'V16-17',
  'V17-',
  'V17'],
 'Ice': ['WI1',
  'WI2-',
  'WI2',
  'WI2+',
  'WI2-3',
  'WI3-',
  'WI3',
  'WI3+',
  'WI3-4',
  'WI4-',
  'WI4',
  'WI4+',
  'WI4-5',
  'WI5-',
  'WI5',
  'WI5+',
  'WI5-6',
  'WI6-',
  'WI6',
  'WI6+',
  'WI6-7',
  'WI7-',
  'WI7',
  'WI7+',
  'WI7-8',
  'WI8-',
  'WI8',
  'AI1',
  'AI1-2',
  'AI2',
  'AI2-3',
  'AI3',
  'AI3-4',
  'AI4',
  'AI4-5',
  'AI5',
  'AI5-6',
  'AI6'],
 'Aid': ['C0',
  'A0',
  'C0+',
  'A0+',
  'C0-1',
  'A0-1',
  'C1-',
  'A1-',
  'C1',
  'A1',
  'C1+',
  'A1+',
  'C1-2',
  'A1-2',
  'C2-',
  'A2-',
  'C2',
  'A2',
  'C2+',
  'A2+',
  'C2-3',
  'A2-3',
  'C3-',
  'A3-',
  'C3',
  'A3',
  'C3+',
  'A3+',
  'C3-4',
  'A3-4',
  'C4-',
  'A4-',
  'C4',
  'A4',
  'C4+',
  'A4+',
  'C4-5',
  'A4-5',
  'C5-',
  'A5-',
  'C5',
  'A5',
  'C5+',
  'A5+'],
 'Mixed': ['M1',
  'M1+',
  'M1-2',
  'M2-',
  'M2',
  'M2+',
  'M2-3',
  'M3-',
  'M3',
  'M3+',
  'M3-4',
  'M4-',
  'M4',
  'M4+',
  'M4-5',
  'M5-',
  'M5',
  'M5+',
  'M5-6',
  'M6-',
  'M6',
  'M6+',
  'M6-7',
  'M7-',
  'M7',
  'M7+',
  'M7-8',
  'M8-',
  'M8',
  'M8+',
  'M8-9',
  'M9-',
  'M9',
  'M9+',
  'M9-10',
  'M10-',
  'M10',
  'M10+',
  'M10-11',
  'M11-',
  'M11',
  'M11+',
  'M12-',
  'M12',
  'M12+',
  'M13-',
  'M13',
  'M13+'],
 'Snow': ['Easy Snow', 'Mod. Snow', 'Steep Snow'],
 'Safety': [
 'PG13',
  'R',
  'X'],
 'Mountaineering': ['I', 'II', 'III', 'IV', 'V', 'VI']}

In [61]:
df = pd.read_csv('clean_data.csv')
pd.set_option('display.max_columns', 50)
df.drop(columns='Unnamed: 0', inplace=True)
df.set_index('id', inplace=True)
df.head()

,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,longitude,latitude,url,Sport,Trad,Boulder,TR,Alpine,Aid,Ice,Snow,Mixed,danger,rope_grade,boulder_grade
id,,,,,,,,,,,,,,,,,,,,,,,,,
105714722,Central Yellow Wall,V3 R,4.4,22,0,South Dakota,Custer State Park,Sylvan Lake,Sylvan Lake Bouldering,Campground Boulder,-103.564,43.8463,https://www.mountainproject.com/route/10571472...,0,0,1,1,0,0,0,0,0,R,0,V3
105714728,Waves,5.8,4.6,217,2,South Dakota,The Needles Of Rushmore,Mount Rushmore National Memorial,South Seas,Shipyard Rock,-103.466,43.8865,https://www.mountainproject.com/route/10571472...,1,0,0,0,0,0,0,0,0,G,5.8,0
105714731,East Chimney Variation,5.7,4.1,40,2,South Dakota,Custer State Park,Cathedral Spires,Station 13,0,-103.534,43.8488,https://www.mountainproject.com/route/10571473...,0,1,0,0,0,0,0,0,0,G,5.7,0
105714734,Conn Diagonal,5.7,4.9,151,3,South Dakota,Custer State Park,Sylvan Lake,Outlets,Outer Outlet,-103.568,43.8487,https://www.mountainproject.com/route/10571473...,0,1,0,0,0,0,0,0,0,G,5.7,0
105714737,Bolts for Bobs,5.8,3.5,125,1,South Dakota,The Needles Of Rushmore,Mount Rushmore National Memorial,South Seas,Borneo,-103.469,43.8870,https://www.mountainproject.com/route/10571473...,1,0,0,0,0,0,0,0,0,G,5.8,0


In [179]:
df.loc[target_id]

name                                                       Leonids
rating                                                         5.9
stars                                                          4.6
starVotes                                                      243
pitches                                                          3
location                                                California
region                                      San Diego and Environs
area                                        South San Diego County
sub_area                                         El Cajon Mountain
wall                                           The Wedge,Left Wall
longitude                                                 -116.882
latitude                                                   32.9127
url              https://www.mountainproject.com/route/10579330...
Sport                                                            1
Trad                                                          

## Get input from user for recommendation

In [178]:
target_id = 105793305
###other parameters to be added here later

### To begin, see if if the climb already exists in db

In [180]:
if target_id in df.index:
    print('We have climb already')
else:
    print('Making API call and Scraping climb data')
    ####call functions to do this

We have climb already


In [181]:
##after all necessary steps target infoshould be loaded into df_target
#we then append this info to our df 
#NOTE index should be id

# df.append(df_target)

## Making Data Numeric

In [182]:
#use df_numeric as we make data numeric
df_numeric = df

#### Catagories to numeric fxn

In [214]:
### make ordinal columns for rope_grade, boulder_grade and danger
def get_rope_grades(grade, catagory):
    for i,match in enumerate(diff_grade[catagory]):
        if match==grade:
            return i+1
    return 0    

#### Make grade columns numeric and add in_range column (defaults to True)

In [215]:
df_numeric['rope_grade']=df['rating'].apply(lambda x: get_rope_grades(x,'Rope'))
df_numeric['boulder_grade']=df['rating'].apply(lambda x: get_rope_grades(x,'Boulder'))
df_numeric['danger']=df['rating'].apply(lambda x: get_rope_grades(x,'Safety'))
df_numeric['in_range'] = 1

df_numeric.head()

,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,longitude,latitude,url,Sport,Trad,Boulder,TR,Alpine,Aid,Ice,Snow,Mixed,danger,rope_grade,boulder_grade,in_range
id,,,,,,,,,,,,,,,,,,,,,,,,,,
105714722,Central Yellow Wall,V3 R,4.4,22,0,South Dakota,Custer State Park,Sylvan Lake,Sylvan Lake Bouldering,Campground Boulder,-103.564,43.8463,https://www.mountainproject.com/route/10571472...,0,0,1,1,0,0,0,0,0,0,0,0,1
105714728,Waves,5.8,4.6,217,2,South Dakota,The Needles Of Rushmore,Mount Rushmore National Memorial,South Seas,Shipyard Rock,-103.466,43.8865,https://www.mountainproject.com/route/10571472...,1,0,0,0,0,0,0,0,0,0,14,0,1
105714731,East Chimney Variation,5.7,4.1,40,2,South Dakota,Custer State Park,Cathedral Spires,Station 13,0,-103.534,43.8488,https://www.mountainproject.com/route/10571473...,0,1,0,0,0,0,0,0,0,0,11,0,1
105714734,Conn Diagonal,5.7,4.9,151,3,South Dakota,Custer State Park,Sylvan Lake,Outlets,Outer Outlet,-103.568,43.8487,https://www.mountainproject.com/route/10571473...,0,1,0,0,0,0,0,0,0,0,11,0,1
105714737,Bolts for Bobs,5.8,3.5,125,1,South Dakota,The Needles Of Rushmore,Mount Rushmore National Memorial,South Seas,Borneo,-103.469,43.8870,https://www.mountainproject.com/route/10571473...,1,0,0,0,0,0,0,0,0,0,14,0,1


## Reccomender

#### Kernel Imports

In [216]:
# Import kernels
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import polynomial_kernel
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import laplacian_kernel
from sklearn.metrics.pairwise import chi2_kernel



In [217]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler

#### Create fxn to see if climb is in search range

In [218]:
#function takes search param range and assigns to original df if climb in_range
def in_range(df_fxn, lat, lon, radius_range=50):
    R= 3958.8 
    if lat == None:
        df_fxn['in_range'] = 1
    else:
        #assign target coords and set to radians for calc
        lat1 = radians(lat)
        lon1 = radians(lon)
        for index, row in df_fxn.iterrows():
            #assign the lat and lon for each climb
            lat2 = radians(row['latitude'])
            lon2 = radians(row['longitude'])

            dlon = lon2 - lon1
            dlat = lat2 - lat1

            a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
            c = 2 * atan2(sqrt(a), sqrt(1 - a))

            distance = R * c
            
            #assign in_range col to 1 if the climb is in range
            if distance < radius_range:
                df_fxn.at[index,'in_range']=1
            else:
                df_fxn.at[index,'in_range']=0   



#### Call function to assign if climb in range

In [219]:
## used to get list of climbs allowed for comparison
in_range(df_numeric, lat = 32.9127, lon = -116.882, radius_range=50)

#### Create df_in_range to run recommender in subset

In [220]:
df_in_range = df_numeric[df_numeric['in_range']==1]
df_in_range.head()

,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,longitude,latitude,url,Sport,Trad,Boulder,TR,Alpine,Aid,Ice,Snow,Mixed,danger,rope_grade,boulder_grade,in_range
id,,,,,,,,,,,,,,,,,,,,,,,,,,
105724579,Bacon Flake,5.8,3.2,69,1,California,Joshua Tree National Park,Echo Rock Area,Echo Cove,Echo Cove - South Face,-117.074,33.0866,https://www.mountainproject.com/route/10572457...,0,1,0,1,0,0,0,0,0,0,14,0,1
105791087,Knob Job,5.8,3.0,63,1,California,San Diego and Environs,South San Diego County,Mission Gorge,Main Wall,-117.051,32.8269,https://www.mountainproject.com/route/10579108...,0,0,0,1,0,0,0,0,0,0,14,0,1
105791105,Hangman's Climb,5.10a,3.6,83,1,California,San Diego and Environs,South San Diego County,Mission Gorge,Main Wall,-117.051,32.8266,https://www.mountainproject.com/route/10579110...,1,0,0,1,0,0,0,0,0,0,19,0,1
105791139,Mariah,5.10a,3.9,54,1,California,San Diego and Environs,South San Diego County,Mission Gorge,Main Wall,-117.051,32.8266,https://www.mountainproject.com/route/10579113...,0,1,0,1,0,0,0,0,0,0,19,0,1
105791566,Suzie's Wild Ride,5.9-,3.6,201,1,California,San Diego and Environs,South San Diego County,Mission Gorge,Main Wall,-117.051,32.8267,https://www.mountainproject.com/route/10579156...,1,0,0,0,0,0,0,0,0,0,16,0,1


### Scale Features

#### Create index df for rec system to reference after running

In [221]:
df_indexed = df_in_range.reset_index()

In [222]:
target_index =df_indexed.index[df_indexed['id']==target_id][0]
target_index

31

In [223]:
df_indexed[df_indexed['id']==target_id]

,id,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,longitude,latitude,url,Sport,Trad,Boulder,TR,Alpine,Aid,Ice,Snow,Mixed,danger,rope_grade,boulder_grade,in_range
31,105793305,Leonids,5.9,4.6,243,3,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"The Wedge,Left Wall",-116.882,32.9127,https://www.mountainproject.com/route/10579330...,1,0,0,0,0,0,0,0,0,0,17,0,1


#### Create Features DF

In [224]:
#creates features df used for comparison
features = df_indexed.loc[:,['stars', 'pitches', 'Sport', 'Trad', 'Boulder', 'TR', 'Alpine', 'Aid',
       'Ice', 'Snow', 'Mixed', 'danger', 'rope_grade', 'boulder_grade']] #,'longitude','latitude',

In [225]:
features.head()

,stars,pitches,Sport,Trad,Boulder,TR,Alpine,Aid,Ice,Snow,Mixed,danger,rope_grade,boulder_grade
0,3.2,1,0,1,0,1,0,0,0,0,0,0,14,0
1,3.0,1,0,0,0,1,0,0,0,0,0,0,14,0
2,3.6,1,1,0,0,1,0,0,0,0,0,0,19,0
3,3.9,1,0,1,0,1,0,0,0,0,0,0,19,0
4,3.6,1,1,0,0,0,0,0,0,0,0,0,16,0


#### Pick scaling type

In [226]:
min_max_scaler = MinMaxScaler()
scalar = StandardScaler()

In [227]:
##### Pick a scaling option ###############################

features_scaled = scalar.fit_transform(features)
# features_scaled = min_max_scaler.fit_transform(features)

##################################################################

#### Add index back into features_scaled Data

In [148]:
# features_scaled=pd.DataFrame(features_scaled)
# features_scaled['id']= pd.Series(features.index, index=features_scaled.index)
# features_scaled.set_index('id',inplace=True)

In [149]:
# features_scaled.index

In [150]:
# features_scaled.values

### Now lets fit the similarity model

#### Rec function

In [151]:
#Construct a reverse map of indices and movie titles
# indices = pd.Series(df.index, index=df['id'])

In [152]:
# df.iloc[0]

In [153]:
# df.index[3]

In [228]:
df_indexed.shape[0]

1022

In [249]:
def get_recommendations(idx, kernel_type):

    # Get the index of the climb that matches the title
#     idx = indices[ids]

    #value to store scores and indicies
    score_matrix = np.ndarray(shape=(len(df_indexed),2), dtype=float)

    #go through the target climb vs all onthers in our db and populate score mtx with index and similarity
    for i in range(df_indexed.shape[0]):
        score = kernel_type(features_scaled[idx].reshape(1,-1),features_scaled[i].reshape(1,-1))
        score_matrix[i][0] =  i        ##the index comparison corresponding to the score
        score_matrix[i][1] = score     ##the score for the current index

    # Sort the climbs based on the similarity scores
    score_matrix = sorted(score_matrix, key=lambda x: x[1], reverse=True)
    
    # score_matrix
#########################  ADD/calculate SIMilarity VALUE   ###########################
    # # Get the scores of the 20 most similar climbs
    score_matrix = score_matrix[1:20]

    # # Get the climb indices (& cast to ints)
    climb_indices = [int(i[0]) for i in score_matrix]
    
    # Return the top 10 most similar movies
#     rec= df['id'].iloc[climb_indices]
    
#     return score_matrix
#     return climb_indices
    return df_indexed.loc[climb_indices,:]

In [ ]:
# def get_recommendations(ids, kernel_type):

#     # Get the index of the climb that matches the title
#     idx = indices[ids]

#     #value to store scores and indicies
#     score_matrix = np.ndarray(shape=(len(df),2), dtype=float)

#     #go through the target climb vs all onthers in our db and populate score mtx with index and similarity
#     for i in range(df.shape[0]):
#         score = kernel_type(features_scaled[idx].reshape(1,-1),features_scaled[i].reshape(1,-1))
#         score_matrix[i][0] = i         ##the index comparison corresponding to the score
#         score_matrix[i][1] = score     ##the score for the current index

#     # Sort the climbs based on the similarity scores
#     score_matrix = sorted(score_matrix, key=lambda x: x[1], reverse=True)
#     # score_matrix

#     # # Get the scores of the 10 most similar climbs
#     score_matrix = score_matrix[1:20]

#     # # Get the climb indices
#     climb_indices = [i[0] for i in score_matrix]
    
#     # Return the top 10 most similar movies
#     rec= df['id'].iloc[climb_indices]
    
#     return df.loc[rec.index,:]
    

#### Call rec fxn

In [ ]:
climb_id =3582 #now target index NOT climb_id number

In [251]:
df[df['name']=='Leonids']

,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,longitude,latitude,url,Sport,Trad,Boulder,TR,Alpine,Aid,Ice,Snow,Mixed,danger,rope_grade,boulder_grade,in_range
id,,,,,,,,,,,,,,,,,,,,,,,,,,
105793305,Leonids,5.9,4.6,243,3,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"The Wedge,Left Wall",-116.882,32.9127,https://www.mountainproject.com/route/10579330...,1,0,0,0,0,0,0,0,0,0,17,0,1


In [ ]:
df[df['id']==climb_id]

In [ ]:
df[(df['rope_grade']==1)]

In [ ]:
df[(df['boulder_grade']==47) & (df['danger']>=2)]

In [252]:
rec=get_recommendations(110, cosine_similarity)
rec

,id,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,longitude,latitude,url,Sport,Trad,Boulder,TR,Alpine,Aid,Ice,Snow,Mixed,danger,rope_grade,boulder_grade,in_range
354,107008221,Soy Chango,5.11c,4.5,22,3,California,San Diego and Environs,South San Diego County,Eagle Peak,Right Wall,-116.706,32.9817,https://www.mountainproject.com/route/10700822...,1,0,0,0,0,0,0,0,0,0,35,0,1
365,107028038,The Direct Route,5.11+,4.8,12,4,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10702803...,1,0,0,0,0,0,0,0,0,0,36,0,1
876,113990601,Holey Moley,5.10d,4.0,1,3,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"The Wedge,Madness Wall",-116.817,32.9047,https://www.mountainproject.com/route/11399060...,1,0,0,0,0,0,0,0,0,0,28,0,1
593,108647367,Tail Tucker Arete,5.11d,4.6,5,4,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10864736...,1,0,0,0,0,0,0,0,0,0,38,0,1
436,107483480,Baby Face,5.11a,4.7,29,4,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10748348...,1,0,0,0,0,0,0,0,0,0,29,0,1
507,108011454,Crazy Horse,5.12a,5.0,1,3,California,San Diego and Environs,South San Diego County,Eagle Peak,Right Wall,-116.706,32.9817,https://www.mountainproject.com/route/10801145...,1,0,0,0,0,0,0,0,0,0,39,0,1
31,105793305,Leonids,5.9,4.6,243,3,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"The Wedge,Left Wall",-116.882,32.9127,https://www.mountainproject.com/route/10579330...,1,0,0,0,0,0,0,0,0,0,17,0,1
517,108056962,Mind Bender,5.12a,4.0,1,3,California,San Diego and Environs,South San Diego County,Eagle Peak,Right Wall,-116.706,32.9817,https://www.mountainproject.com/route/10805696...,1,0,0,0,0,0,0,0,0,0,39,0,1
873,113969763,The Shining Slab,5.11b,4.0,3,4,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/11396976...,1,0,0,0,0,0,0,0,0,0,32,0,1
416,107418139,The Commander,5.12a,4.8,20,5,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"The Wedge,Center/Headwall",-116.818,32.9046,https://www.mountainproject.com/route/10741813...,1,0,0,0,0,0,0,0,0,0,39,0,1


In [248]:
cosine_similarity(features_scaled[target_index].reshape(1,-1),features_scaled[898].reshape(1,-1))

array([[0.94823005]])

In [253]:
rec=get_recommendations(target_index, cosine_similarity)
rec

,id,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,longitude,latitude,url,Sport,Trad,Boulder,TR,Alpine,Aid,Ice,Snow,Mixed,danger,rope_grade,boulder_grade,in_range
110,105835122,Triton Tower,5.11a,4.5,68,3,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"The Wedge,Triton Tower",-116.818,32.9043,https://www.mountainproject.com/route/10583512...,1,0,0,0,0,0,0,0,0,0,29,0,1
436,107483480,Baby Face,5.11a,4.7,29,4,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10748348...,1,0,0,0,0,0,0,0,0,0,29,0,1
876,113990601,Holey Moley,5.10d,4.0,1,3,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"The Wedge,Madness Wall",-116.817,32.9047,https://www.mountainproject.com/route/11399060...,1,0,0,0,0,0,0,0,0,0,28,0,1
365,107028038,The Direct Route,5.11+,4.8,12,4,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10702803...,1,0,0,0,0,0,0,0,0,0,36,0,1
437,107491542,Sunset Streaks,5.10a,4.4,85,5,California,San Diego and Environs,South San Diego County,Corte Madera,Main Wall,-117.140,32.9126,https://www.mountainproject.com/route/10749154...,1,0,0,0,0,0,0,0,0,0,19,0,1
354,107008221,Soy Chango,5.11c,4.5,22,3,California,San Diego and Environs,South San Diego County,Eagle Peak,Right Wall,-116.706,32.9817,https://www.mountainproject.com/route/10700822...,1,0,0,0,0,0,0,0,0,0,35,0,1
356,107008761,Cruise Line,5.10c,4.4,63,5,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10700876...,1,0,0,0,0,0,0,0,0,0,25,0,1
593,108647367,Tail Tucker Arete,5.11d,4.6,5,4,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10864736...,1,0,0,0,0,0,0,0,0,0,38,0,1
898,114401732,Shiver,5.7+,4.0,2,2,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"Mountaineers Wall,The Toe",-116.826,32.8967,https://www.mountainproject.com/route/11440173...,1,0,0,0,0,0,0,0,0,0,12,0,1
367,107028785,Up Stream,5.10c,4.4,14,2,California,San Diego and Environs,South San Diego County,Eagle Peak,Right Wall,-116.706,32.9817,https://www.mountainproject.com/route/10702878...,1,0,0,0,0,0,0,0,0,0,25,0,1


In [254]:
rec=get_recommendations(target_index, rbf_kernel)
rec

,id,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,longitude,latitude,url,Sport,Trad,Boulder,TR,Alpine,Aid,Ice,Snow,Mixed,danger,rope_grade,boulder_grade,in_range
110,105835122,Triton Tower,5.11a,4.5,68,3,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"The Wedge,Triton Tower",-116.818,32.9043,https://www.mountainproject.com/route/10583512...,1,0,0,0,0,0,0,0,0,0,29,0,1
876,113990601,Holey Moley,5.10d,4.0,1,3,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"The Wedge,Madness Wall",-116.817,32.9047,https://www.mountainproject.com/route/11399060...,1,0,0,0,0,0,0,0,0,0,28,0,1
354,107008221,Soy Chango,5.11c,4.5,22,3,California,San Diego and Environs,South San Diego County,Eagle Peak,Right Wall,-116.706,32.9817,https://www.mountainproject.com/route/10700822...,1,0,0,0,0,0,0,0,0,0,35,0,1
367,107028785,Up Stream,5.10c,4.4,14,2,California,San Diego and Environs,South San Diego County,Eagle Peak,Right Wall,-116.706,32.9817,https://www.mountainproject.com/route/10702878...,1,0,0,0,0,0,0,0,0,0,25,0,1
109,105835105,Mañana,5.10,4.1,42,2,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"The Wedge,Left Wall",-116.818,32.9046,https://www.mountainproject.com/route/10583510...,1,0,0,0,0,0,0,0,0,0,23,0,1
898,114401732,Shiver,5.7+,4.0,2,2,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"Mountaineers Wall,The Toe",-116.826,32.8967,https://www.mountainproject.com/route/11440173...,1,0,0,0,0,0,0,0,0,0,12,0,1
436,107483480,Baby Face,5.11a,4.7,29,4,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10748348...,1,0,0,0,0,0,0,0,0,0,29,0,1
507,108011454,Crazy Horse,5.12a,5.0,1,3,California,San Diego and Environs,South San Diego County,Eagle Peak,Right Wall,-116.706,32.9817,https://www.mountainproject.com/route/10801145...,1,0,0,0,0,0,0,0,0,0,39,0,1
372,107047681,Lucky Stars,5.11c,3.7,3,3,California,San Diego and Environs,South San Diego County,Eagle Peak,Right Wall,-116.706,32.9817,https://www.mountainproject.com/route/10704768...,1,0,0,0,0,0,0,0,0,0,35,0,1
517,108056962,Mind Bender,5.12a,4.0,1,3,California,San Diego and Environs,South San Diego County,Eagle Peak,Right Wall,-116.706,32.9817,https://www.mountainproject.com/route/10805696...,1,0,0,0,0,0,0,0,0,0,39,0,1


In [255]:
rec=get_recommendations(target_index, laplacian_kernel)
rec

,id,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,longitude,latitude,url,Sport,Trad,Boulder,TR,Alpine,Aid,Ice,Snow,Mixed,danger,rope_grade,boulder_grade,in_range
110,105835122,Triton Tower,5.11a,4.5,68,3,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"The Wedge,Triton Tower",-116.818,32.9043,https://www.mountainproject.com/route/10583512...,1,0,0,0,0,0,0,0,0,0,29,0,1
354,107008221,Soy Chango,5.11c,4.5,22,3,California,San Diego and Environs,South San Diego County,Eagle Peak,Right Wall,-116.706,32.9817,https://www.mountainproject.com/route/10700822...,1,0,0,0,0,0,0,0,0,0,35,0,1
876,113990601,Holey Moley,5.10d,4.0,1,3,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"The Wedge,Madness Wall",-116.817,32.9047,https://www.mountainproject.com/route/11399060...,1,0,0,0,0,0,0,0,0,0,28,0,1
367,107028785,Up Stream,5.10c,4.4,14,2,California,San Diego and Environs,South San Diego County,Eagle Peak,Right Wall,-116.706,32.9817,https://www.mountainproject.com/route/10702878...,1,0,0,0,0,0,0,0,0,0,25,0,1
507,108011454,Crazy Horse,5.12a,5.0,1,3,California,San Diego and Environs,South San Diego County,Eagle Peak,Right Wall,-116.706,32.9817,https://www.mountainproject.com/route/10801145...,1,0,0,0,0,0,0,0,0,0,39,0,1
436,107483480,Baby Face,5.11a,4.7,29,4,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10748348...,1,0,0,0,0,0,0,0,0,0,29,0,1
109,105835105,Mañana,5.10,4.1,42,2,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"The Wedge,Left Wall",-116.818,32.9046,https://www.mountainproject.com/route/10583510...,1,0,0,0,0,0,0,0,0,0,23,0,1
517,108056962,Mind Bender,5.12a,4.0,1,3,California,San Diego and Environs,South San Diego County,Eagle Peak,Right Wall,-116.706,32.9817,https://www.mountainproject.com/route/10805696...,1,0,0,0,0,0,0,0,0,0,39,0,1
898,114401732,Shiver,5.7+,4.0,2,2,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"Mountaineers Wall,The Toe",-116.826,32.8967,https://www.mountainproject.com/route/11440173...,1,0,0,0,0,0,0,0,0,0,12,0,1
752,110544594,Comply or Else,5.10b,3.0,1,3,California,San Diego and Environs,South San Diego County,Eagle Peak,The Summit,-116.708,32.9823,https://www.mountainproject.com/route/11054459...,1,0,0,0,0,0,0,0,0,0,22,0,1


In [256]:
rec=get_recommendations(target_index, euclidean_distances)
rec

,id,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,longitude,latitude,url,Sport,Trad,Boulder,TR,Alpine,Aid,Ice,Snow,Mixed,danger,rope_grade,boulder_grade,in_range
20,105792322,Lie Detector,5.12a/b C1,4.1,37,1,California,San Diego and Environs,North San Diego County,Mount Woodson,Robbins Boulder,-116.966,33.0056,https://www.mountainproject.com/route/10579232...,0,1,0,1,0,1,0,0,0,0,0,0,1
10,105791615,Hear My Train A Comin',5.11d C1,4.4,34,1,California,San Diego and Environs,North San Diego County,Mount Woodson,Hear My Train A Comin' Boulder,-116.961,33.0064,https://www.mountainproject.com/route/10579161...,0,1,0,1,0,1,0,0,0,0,0,0,1
1009,117849416,Wall of the Unclimbables,5.10+ C0,3.8,4,1,California,San Diego and Environs,South San Diego County,Otay Mesa,0,-116.907,32.6030,https://www.mountainproject.com/route/11784941...,0,1,0,0,0,1,0,0,0,0,0,0,1
406,107267134,Where's The Breeze?,A2 R,4.0,4,1,California,San Diego and Environs,South San Diego County,McCain Valley,Lowenbrau Pinnacle,-116.280,32.7522,https://www.mountainproject.com/route/10726713...,0,1,0,0,0,1,0,0,0,0,0,0,1
28,105792835,Clockwork Orange Seam,A3,4.3,3,1,California,San Diego and Environs,North San Diego County,Mount Woodson,Rockwork Orange Boulder,-116.970,33.0057,https://www.mountainproject.com/route/10579283...,0,1,0,0,0,1,0,0,0,0,0,0,1
405,107267127,Upper Air,A2,3.0,2,1,California,San Diego and Environs,South San Diego County,McCain Valley,Lowenbrau Pinnacle,-116.280,32.7522,https://www.mountainproject.com/route/10726712...,0,0,0,0,0,1,0,0,0,0,0,0,1
434,107473844,Crack Up,C2,3.8,4,1,California,San Diego and Environs,South San Diego County,McCain Valley,Lowenbrau Pinnacle,-116.280,32.7522,https://www.mountainproject.com/route/10747384...,0,0,0,0,0,1,0,0,0,0,0,0,1
454,107595769,Bat Hooks Gone Wild,C2+,4.5,6,1,California,San Diego and Environs,South San Diego County,El Cajon Mountain,Quarry Wall,-116.825,32.8927,https://www.mountainproject.com/route/10759576...,0,0,0,0,0,1,0,0,0,0,0,0,1
983,117008483,TFP,C2+,4.7,3,1,California,San Diego and Environs,South San Diego County,El Cajon Mountain,Quarry Wall,-116.825,32.8927,https://www.mountainproject.com/route/11700848...,0,0,0,0,0,1,0,0,0,0,0,0,1
218,106199313,Super Air,A1-,2.9,7,1,California,San Diego and Environs,South San Diego County,McCain Valley,Lowenbrau Pinnacle,-116.280,32.7522,https://www.mountainproject.com/route/10619931...,1,0,0,0,0,1,0,0,0,0,0,0,1


In [257]:
rec=get_recommendations(target_index, linear_kernel)
rec

,id,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,longitude,latitude,url,Sport,Trad,Boulder,TR,Alpine,Aid,Ice,Snow,Mixed,danger,rope_grade,boulder_grade,in_range
410,107382396,Diamond of San Diego,5.11d,4.4,10,5,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"The Wedge,Center/Headwall",-116.902,32.8427,https://www.mountainproject.com/route/10738239...,1,0,0,0,0,0,0,0,0,0,38,0,1
356,107008761,Cruise Line,5.10c,4.4,63,5,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10700876...,1,0,0,0,0,0,0,0,0,0,25,0,1
478,107917616,Southwest Face,5.11b,4.2,6,5,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10791761...,1,0,0,0,0,0,0,0,0,0,32,0,1
437,107491542,Sunset Streaks,5.10a,4.4,85,5,California,San Diego and Environs,South San Diego County,Corte Madera,Main Wall,-117.140,32.9126,https://www.mountainproject.com/route/10749154...,1,0,0,0,0,0,0,0,0,0,19,0,1
689,109721861,Friendly Fire,5.12a,4.0,3,5,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10972186...,1,0,0,0,0,0,0,0,0,0,39,0,1
96,105813804,Cause for Pause,5.10c,3.7,21,5,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"The Wedge,Left Wall",-116.818,32.9046,https://www.mountainproject.com/route/10581380...,1,0,0,0,0,0,0,0,0,0,25,0,1
373,107048594,Rock Jungle,5.10b,3.6,16,5,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10704859...,1,0,0,0,0,0,0,0,0,0,22,0,1
365,107028038,The Direct Route,5.11+,4.8,12,4,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10702803...,1,0,0,0,0,0,0,0,0,0,36,0,1
593,108647367,Tail Tucker Arete,5.11d,4.6,5,4,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10864736...,1,0,0,0,0,0,0,0,0,0,38,0,1
436,107483480,Baby Face,5.11a,4.7,29,4,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10748348...,1,0,0,0,0,0,0,0,0,0,29,0,1


In [258]:
rec=get_recommendations(target_index, polynomial_kernel)
rec

,id,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,longitude,latitude,url,Sport,Trad,Boulder,TR,Alpine,Aid,Ice,Snow,Mixed,danger,rope_grade,boulder_grade,in_range
410,107382396,Diamond of San Diego,5.11d,4.4,10,5,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"The Wedge,Center/Headwall",-116.902,32.8427,https://www.mountainproject.com/route/10738239...,1,0,0,0,0,0,0,0,0,0,38,0,1
356,107008761,Cruise Line,5.10c,4.4,63,5,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10700876...,1,0,0,0,0,0,0,0,0,0,25,0,1
478,107917616,Southwest Face,5.11b,4.2,6,5,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10791761...,1,0,0,0,0,0,0,0,0,0,32,0,1
437,107491542,Sunset Streaks,5.10a,4.4,85,5,California,San Diego and Environs,South San Diego County,Corte Madera,Main Wall,-117.140,32.9126,https://www.mountainproject.com/route/10749154...,1,0,0,0,0,0,0,0,0,0,19,0,1
689,109721861,Friendly Fire,5.12a,4.0,3,5,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10972186...,1,0,0,0,0,0,0,0,0,0,39,0,1
96,105813804,Cause for Pause,5.10c,3.7,21,5,California,San Diego and Environs,South San Diego County,El Cajon Mountain,"The Wedge,Left Wall",-116.818,32.9046,https://www.mountainproject.com/route/10581380...,1,0,0,0,0,0,0,0,0,0,25,0,1
373,107048594,Rock Jungle,5.10b,3.6,16,5,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10704859...,1,0,0,0,0,0,0,0,0,0,22,0,1
365,107028038,The Direct Route,5.11+,4.8,12,4,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10702803...,1,0,0,0,0,0,0,0,0,0,36,0,1
593,108647367,Tail Tucker Arete,5.11d,4.6,5,4,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10864736...,1,0,0,0,0,0,0,0,0,0,38,0,1
436,107483480,Baby Face,5.11a,4.7,29,4,California,San Diego and Environs,South San Diego County,Eagle Peak,Left Wall,-116.706,32.9815,https://www.mountainproject.com/route/10748348...,1,0,0,0,0,0,0,0,0,0,29,0,1


In [259]:
rec=get_recommendations(climb_id, sigmoid_kernel)
rec

NameError: name 'climb_id' is not defined

In [ ]:
rec=get_recommendations(climb_id, chi2_kernel)
rec